In [1]:
import numpy as np
%matplotlib inline
import pandas as pd
import gseapy as gp
import matplotlib.pyplot as plt
import math

## Enrichr: http://amp.pharm.mssm.edu/Enrichr/#

In [25]:
names = gp.get_library_name()
names[:3]

[u'ARCHS4_Cell-lines', u'ARCHS4_IDG_Coexp', u'ARCHS4_Kinases_Coexp']

In [16]:
gene_list = ['CTLA2B', 'SCARA3', 'LOC100044683', 'CMBL', 'CLIC6', 'IL13RA1', 'TACSTD2', 'DKKL1', 'CSF1', 'CITED1']
gene_rank = 3*np.random.random(len(gene_list))
rnk = pd.DataFrame(np.array([gene_list, gene_rank]).T, columns = ['gene', 'score'])
rnk

#rnk = pd.DataFrame(np.array([gene_list]).T)
#rnk

,gene,score
0,CTLA2B,0.559979116181
1,SCARA3,0.35091183262
2,LOC100044683,1.18096971413
3,CMBL,1.36634783039
4,CLIC6,0.440883203891
5,IL13RA1,2.08208970422
6,TACSTD2,0.369493901287
7,DKKL1,0.874569977306
8,CSF1,1.73263056522
9,CITED1,2.65293078095


In [12]:
#enr = gp.enrichr(gene_list=gene_list, description='pathway', gene_sets='KEGG_2016', outdir='test', cutoff=0.05, format='png')
enr = gp.enrichr(gene_list=gene_list, description='pathway', gene_sets='KEGG_2016', outdir='test', cutoff=0.05, format='png')
enr.res2d.iloc[:,:]

2017-11-13 18:45:33,938 Warning: No enrich terms using library KEGG_2016 when cuttoff = 0.05


,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Z-score,Combined Score,Genes
0,Cytokine-cytokine receptor interaction_Homo sa...,2/265,0.007337,0.073374,0.004224,0.042236,-1.877182,9.225912,CSF1;IL13RA1
1,Hematopoietic cell lineage_Homo sapiens_hsa04640,1/88,0.043148,0.127111,0.037596,0.110826,-1.757680,5.524578,CSF1
2,TNF signaling pathway_Homo sapiens_hsa04668,1/110,0.053670,0.127111,0.046742,0.110826,-1.833875,5.363892,CSF1
3,Rheumatoid arthritis_Homo sapiens_hsa05323,1/90,0.044109,0.127111,0.038430,0.110826,-1.717461,5.360340,CSF1
4,Osteoclast differentiation_Homo sapiens_hsa04380,1/132,0.064088,0.127111,0.055830,0.110826,-1.760622,4.837309,CSF1
5,Jak-STAT signaling pathway_Homo sapiens_hsa04630,1/158,0.076266,0.127111,0.066495,0.110826,-1.803638,4.641702,IL13RA1
6,Rap1 signaling pathway_Homo sapiens_hsa04015,1/211,0.100651,0.134873,0.087987,0.118012,-1.810597,4.157300,CSF1
7,Ras signaling pathway_Homo sapiens_hsa04014,1/227,0.107898,0.134873,0.094410,0.118012,-1.827518,4.069093,CSF1
8,PI3K-Akt signaling pathway_Homo sapiens_hsa04151,1/341,0.158029,0.175587,0.139300,0.154778,-1.859626,3.430970,CSF1
9,Metabolic pathways_Homo sapiens_hsa01100,1/1239,0.472533,0.472533,0.442232,0.442232,-1.788092,1.340439,CMBL


## Prerank: http://software.broadinstitute.org/cancer/software/genepattern/modules/docs/GSEAPreranked/1

In [17]:
pre_res = gp.prerank(rnk=rnk, gene_sets='KEGG_2016', outdir='prerank_report',format='png')
pre_res.res2d.head()

2017-11-13 18:48:21,979 No gene sets passed throught filtering condition!!!, try new paramters again!
Note: Gene names for gseapy is case sensitive.


SystemExit: 1

## Cancer gene Census (Controllability Paper)

In [32]:
cancer_gene_census = pd.read_csv("cancer_gene_census.csv")
gene_symbols = cancer_gene_census.loc[:,"Gene Symbol"]
gene_names = cancer_gene_census.loc[:,"Name"]
gene_string = []
list_of_synonyms = cancer_gene_census.loc[:,"Synonyms"]
for synonyms in list_of_synonyms:
    if str(synonyms) != 'nan':        
        for synonym in synonyms.split(','):
            if synonym[:3] == "ENS":
                gene_string.append(synonym)
print "Number of genes = ", len(gene_symbols)

Number of genes =  699


In [31]:
#with open("mimTitles.txt", "r") as f:
gene_entrez = []
gene_symbols = []
gene_string = []
with open("mim2gene.txt") as f:
    for line in f:
        if len(line.split('\t'))>1:
            if line.split('\t')[1] == "gene":
                if len(line.split('\t'))>2:
                    gene_entrez.append(line.split('\t')[2])
                    gene_symbols.append(line.split('\t')[3])
                    gene_string.append(line.split('\t')[4])

print "Nb of genes =", len(gene_string)

Nb of genes = 15766
